# 3D Geospatial Mapping with PyDeck

PyDeck is an interactive spatial data visualization library in Python. Powered by deck.gl, it can quickly execute high-precision mapping for a large dataset and integrate with commonly used map providers such as Google Maps, Mapbox, and more. It takes a “layer” approach when creating visualizations, making it easy to compile, test, and reuse.

This tutorial will walk you through how to make two 3D visualization in the sidewalk shed story. One for sidewalk sheds' proximity with stores, the other for the estimated cost level of sidewalk sheds. Generally speaking, there are a few steps to be completed:
1. Install and import PyDeck: only need to do this once in a notebook.
2. Import data
3. Process data (optional)
4. Configure PyDeck layers
5. Add layer(s) to Deck
6. Export to .html file

Rather than trying to improve upon existing tutorials of PyDeck (which there are many online), this tutorial attempts to be a reference for using PyDeck for journalistic data visualization practices with a short turnaround time.

## Graph 1: Sidewalk sheds and stores

### Install and import PyDeck
In the Python environment, install PyDeck and import it (as well as other required library).

In [ ]:
!pip install pydeck
import pydeck as pdk
import pandas as pd

### Import data
1. Sidewalk shed permit data
Import the datast you need with pandas. Here, we need all sidewalk shed permits data, 

In [ ]:
AP = pd.read_csv('All_Shed_Permits.csv')

### Data Processing
Then, filter to active sidewalk shed permits only. 

We also need to compute the age of each sidewalk shed as the difference between their creation dates and Feb 01, 2024.

**IMPORTANT** ❗️: **Make sure that there is no *null* value in your dataset to prevent error in PyDeck.**

In [ ]:
# Filter to active shed permits only
AP = AP[AP['active'] == True]

# Compute age of each sidewalk shed
AP['created_date'] = pd.to_datetime(AP['created_date'], format='mixed')
cur_date = pd.to_datetime('2024-02-01')
AP['Age'] = (cur_date - AP['created_date']).dt.days.astype(float)

# Drop Null in the dataset
AP= AP[(AP['c_b_no'].isna() == False) & (AP['lat'].isna() == False) & (AP['long'].isna() == False)]


AP = AP[['job__', 'Age', 'lat', 'long', 'c_b_no']]

2. Store front vacancy data

Similarly, import the store front vacancy data with information about all non-empty store in NYC.

In [ ]:
RS = pd.read_csv('Retail_Stores.csv')
RS = RS[['PROPERTY.STREET.ADDRESS.OR.STOREFRONT.ADDRESS','LATITUDE', 'LONGITUDE', 'PRIMARY.BUSINESS.ACTIVITY', 'COMMUNITY.BOARD']]

This dataset is already cleaned so that there is no null value. But bonus point if you could filter the 'LATITUDE' and 'LONGITUDE' column with is.na()


In [ ]:
# Your code here!

### Initialize a ViewState
To use PyDeck, first initialize a ViewState, which is where the screen is focused as the file is opened.

A few parameters to know:

- **latitute, longitude**: where the initial focus position is

- **zoom**: magnification level of the map (between 0 for the whole world and 24 for individual buildings)

- **pitch**: up/down angle relative to the map’s plane (Angle in degrees)

- **bearing**: the left/right angle relative to the map’s north (Angle in degree)

Change these parameter based on what you'd like to show. 

As the scope of this story is in New York City, I changed the initial focus position to the Midtown area of Manhattan, where a number of sidewalk sheds are still active, by setting the **latitude** and **longitude**. 


In [ ]:
INITIAL_VIEW_STATE = pdk.ViewState(latitude=40.801412, 
                                   longitude=-73.967279, 
                                   zoom=14, max_zoom=16, pitch=60, bearing=0)

### Configure layers by setting parameters
Then, create the visualization by each layer that you would like to use. For any layer, one must specify:
- **a unique layer name** 
- **the layer type**

    There are a bunch of different 2D and 3D layers for different purposes. All preset for you to grab and use. Choose the one that fits your story the most with examples from the "[Layer Catalog Overview](https://deck.gl/docs/api-reference/layers)".

- **the dataset used in this layer**

and any customizable parameters to this layer, including:

- **get_position**: positions of columns. Usually latitute and longitude.

- **get_fill_color**:  color of columns. Could be one [r,g,b,a] value for all or different colors for each columns

- **radius**: the radius of columns

- **pickable**: "True" or "False". Whether columns are interactable

- **auto_highlight**: "True" or "False". When columns are pickable, whether the color changes as hovering

- others...

For example, use the "ColumnLayer" to visualize all the active sidewalk shed permits in NYC. Specify the layer name, dataset to use, position of each column, and other information in the "**Layer**" class. 

In [ ]:
Sheds = pdk.Layer(
    "ColumnLayer",                      # Layer name
    AP,                                 # Dataframe
    get_position=["long", "lat"],       # Position of the columns       
    get_elevation=100,                   # Height of the columns
    opacity = 0.5,                      # Opacity                      
    get_fill_color=[250,0,0,255],       # Color of the columns: [r,g,b,a]
    radius=12,                          # Radius of the columns
    pickable=False,                     # Whether the columns are interactable
    auto_highlight=False,                # When columns are pickable, whether the color changes as hovering
)

### Add layers to Deck
To add one or more layers into the graph, change the “layers” parameters in [Deck](https://deckgl.readthedocs.io/en/latest/deck.html) class. To avoid errors, a layer must be declared before being used. Likewise, initial_view_state should also take a declared view state.

The style of the base map can also be chosen from ‘light’, ‘dark’, ‘road’, ‘satellite’, ‘dark_no_labels’, and ‘light_no_labels’ by changing the map_style parameter.

In [ ]:
r = pdk.Deck(layers=[Sheds], map_style = "light", initial_view_state=INITIAL_VIEW_STATE)

### Export to .html file
Finally, use to_html to save the visualization as a .html file. The file should be saved under the same directory as this notebook. Open it in your browser to see how it looks!

In [ ]:
r.to_html("Sheds.html")

*Try it yourself*: Can you add a tooltip to the visualization so that it shows more information when hovering over the columns?

Hint: change the pickable parameter in layer

In [ ]:
# Your code here!
# Or you can also change the parameter in the cell above

Now, we've completed the first round of 3D geographical visualization, which gives us an interactive html that in which we can zoom in and out, pan, and hover over data points. 

But instead of showing only the position, we should also leverage the other dimensions in 3D visualizaiton. In the next round of visualization, we'll use the **height** of each column to represent the age of a sidewalk shed and the **color** to show whether it is proximate to stores.

### Data Processing
Use the same active sidewalk shed permit dataset and calculate the normalized height with the "Age".

In [ ]:
def AgeHeight(age, mini, maxi):
    return (age - mini) / (maxi-mini) * 1000

AP['Height'] = AP['Age'].apply(lambda x: AgeHeight(x, AP['Age'].min(), AP['Age'].max()))

Next, compute 'Color' with the distance between a sidewalk shed and retail stores. We first define a function to calculate the distance between two geographical coordinates. Add a "Color" column for active sidewalk sheds with gray [210,196,174,200] as the default color.

In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in meters
    R = 6371000
    # Convert coordinates from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    # Haversine formula
    a = math.sin(delta_phi / 2.0) ** 2 + \
        math.cos(phi1) * math.cos(phi2) * \
        math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Output distance in meters
    return distance

AP['Around_Store'] = False
AP['Store_Num'] = 0
RS['IMPACTED'] = False

for index, row in AP.iterrows():
    shed_cb = row['c_b_no']
    shed_lat = row['lat']
    shed_long = row['long']
    if(RS[RS['COMMUNITY.BOARD'] == shed_cb].empty == False):
        for a, b in RS[RS['COMMUNITY.BOARD'] == shed_cb].iterrows():
            store_lat = b['LATITUDE']
            store_long = b['LONGITUDE']
            if(haversine(shed_lat, shed_long, store_lat, store_long)<=35):
                AP.at[index, 'Around_Store'] = True
                AP.at[index, 'Store_Num']+=1
                RS.at[a, 'IMPACTED'] = True

For each sidewalk shed permits in the data, calculate its distance with stores in the same community board. If any of their distances is smaller than 35 meters, the sidewalk shed is proximate to a business, and therefore we change its "Color" to orange [242,131,55,255].

In [ ]:
# Orange & Gray
# Gray as the default
AP['Color'] = AP['Age'].apply(lambda x: [210,196,174,200])

AP['Color'] = AP['Around'].apply(lambda x: [242,131,55,255] if x else [210,196,174,255])

### Configure layers by setting parameters
We define a new layer and specify additional parameters in a simiar way. Instead of one value, we use column "Height for “get_elevation” and "Color" for “get_fill_color”

In [ ]:
Sheds2 = pdk.Layer(
    "ColumnLayer",
    AP,
    get_position=["long", "lat"],
    get_elevation='Height',             # Heights represent the age of sidewalk sheds
    opacity = 0.5,
    elevation_scale=1,  
    get_fill_color="Color",             # Color means whether there are stores around               
    radius=12, 
    pickable=False,
    auto_highlight=False,
)

After configuring a new layer, we need to add layer to Deck and export to .html file. Could you do it on your own.
Hint: refer to the code above


In [ ]:
# Your code here!


Here's how to do it:

In [ ]:
r = pdk.Deck(layers=[Sheds2], map_style = "light", initial_view_state=INITIAL_VIEW_STATE)
r.to_html("Sheds2.html")

Next, we can also add more contextual information by visualizing the locations of both sidewalk sheds and retail stores on a map. Visualizing thousands of data points or two distinct types would be limiting on a 2D graph without potentially cluttering the design’s single perspective, negatively impacting how clearly it can visually communicate key takeaways. Thus, use 3D blocks for sidewalk sheds and 2D circles for retail stores to show the extensive scale of sidewalk sheds’ impacts while preventing the overlapping problem that clutters readability.

Since we already have all the information required in the store data(position), we don't need any data processing at this moment.

### Configure layers by setting parameters
We use 2D circles to show the position of retail stores with the layer "ScatterplotLayer". Set the position parameter for stores with their longitude and latitude. Other parameters are arbitraty in this case. Since these are 2D circles, there is no height for this layer.

In [ ]:
Stores = pdk.Layer(
    'ScatterplotLayer',
    RS,
    get_position=['LONGITUDE', 'LATITUDE'],
    get_radius=20,
    get_fill_color = [161,218,180,255],
    get_line_color = [0,0,0,255],
    get_line_width = 20,
    pickable=False,
    auto_highlight=False
)

### Add layers to Deck
One advantage of PyDeck is that each layer is separate. One could initialize more than one layer respectively and add multiple layers into one visualization altogether. 

In [ ]:
r = pdk.Deck(layers=[Sheds2, Stores], map_style = "light", initial_view_state=INITIAL_VIEW_STATE)

### Export to .html file

In [ ]:
r.to_html("Shed_store.html")

## Graph 2. Sidewalk shed estimated costs
### Import data

In [ ]:
# Job Number; Sidewalk Shed/Linear Feet
length = pd.read_csv('Active_Sheds2.csv')
length2 = length[["Job Number", "Sidewalk Shed/Linear Feet", "Age", "Latitude Point", "Longitude Point"]]
length2.loc[length2['Sidewalk Shed/Linear Feet'] == 0] = 35

### Compute data
Compute cost: 
installment cost = sidewalk shed linear feet * 115 (unit price)
total cost = installment cost + consecutive month * 10% * installment cost

In [ ]:
length2['month'] = length['Age'] / 30 - 1
length2.loc[length2['month'] <0,]['month'] = 0
length2['first_cost'] = length2["Sidewalk Shed/Linear Feet"] * 115
length2['rent'] = length2['first_cost'] * 0.1
length2['cost'] = length2['first_cost'] + length2['rent'] * length2['month']
length2['lng'] = length2['Longitude Point']
length2['lat'] = length2['Latitude Point']

Compute height

In [ ]:
length2['Height'] = length2['Age'].apply(lambda x: AgeHeight(x, length2['Age'].min(), length2['Age'].max()))

Compute color

In [ ]:
def ColorRange(cost, colors):
    if(cost < 1000): return colors[0]
    if(cost < 10000): return colors[1]
    if(cost < 100000): return colors[2]
    if(cost < 500000): return colors[3]
    return colors[4]
 

cost_color2 = [[222,203,228, 255], [204,235,197, 255], [179,205,227, 255], [254,217,166, 255], [251,180,174, 255]]

cost_color = [[152,78,163, 255], [55,126,184, 255], [77,175,74, 255], [255,127,0, 255], [228,26,28, 255]]

length2['Color'] = length2['cost'].apply(lambda x: ColorRange(x, cost_color2))

### Configure layers by setting parameters

In [ ]:
Costs = pdk.Layer(
    "ColumnLayer",
    length2[['cost', 'lng', 'lat', 'Color', 'Age', 'Height']],
    get_position=["lng", "lat"],
    get_elevation='Height',
    opacity = 1,
    elevation_scale=1,  
    get_fill_color="Color",  
    radius=15,  
    pickable=True,
    auto_highlight=True,
)

### Add layers to Deck

In [ ]:
r = pdk.Deck(layers=Costs, map_style = "light", initial_view_state=INITIAL_VIEW_STATE)

### Export to .html file

In [ ]:
r.to_html("Sheds_cost_length.html")

Nice job! Now you know the basics of PyDeck (and a lot about sidewalk sheds)

Feel free to go to the template notebook and play with other data!